In [ ]:
# Importações para representação de texto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Word Embeddings
from gensim.models import Word2Vec, FastText
from gensim.models.keyedvectors import KeyedVectors

# Visualização
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

# NLP
import nltk
from nltk.tokenize import word_tokenize

# Datasets
import sys
sys.path.append('..')
from datasets.textos_exemplo import *
from utils.nlp_utils import *

print("🔤 Bibliotecas para representação de texto carregadas!")


In [ ]:
# Preparar dados para representação
corpus_pequeno = [
    "O gato subiu no telhado",
    "O cachorro correu no parque", 
    "O gato desceu do telhado",
    "O cachorro brincou no parque com outros cachorros",
    "Os gatos e cachorros são animais domésticos"
]

print("📝 Corpus de exemplo:")
for i, doc in enumerate(corpus_pequeno):
    print(f"{i+1}. {doc}")

print("\n" + "="*60)

# 1. Bag of Words Binário
print("🎒 1. Bag of Words BINÁRIO")
bow_binary = CountVectorizer(binary=True, lowercase=True)
bow_binary_matrix = bow_binary.fit_transform(corpus_pequeno)

# 2. Bag of Words por Contagem
print("\n🎒 2. Bag of Words por CONTAGEM")
bow_count = CountVectorizer(binary=False, lowercase=True)
bow_count_matrix = bow_count.fit_transform(corpus_pequeno)

# 3. TF-IDF
print("\n🎒 3. TF-IDF")
tfidf = TfidfVectorizer(lowercase=True)
tfidf_matrix = tfidf.fit_transform(corpus_pequeno)

# Função para mostrar matriz de forma legível
def mostrar_matriz(matrix, vectorizer, titulo, top_features=10):
    print(f"\n{titulo}")
    print("-" * len(titulo))
    
    # Obter nomes das features
    feature_names = vectorizer.get_feature_names_out()
    
    # Converter para array denso
    dense_matrix = matrix.toarray()
    
    # Criar DataFrame para visualização
    df = pd.DataFrame(dense_matrix, 
                      columns=feature_names,
                      index=[f"Doc {i+1}" for i in range(len(corpus_pequeno))])
    
    print(f"📊 Dimensões: {matrix.shape[0]} documentos × {matrix.shape[1]} features")
    print(f"🔍 Esparsidade: {(1.0 - matrix.nnz / (matrix.shape[0] * matrix.shape[1])):.3f}")
    
    # Mostrar apenas as top features para visualização
    if len(feature_names) > top_features:
        # Selecionar features com maior variância
        feature_variance = np.var(dense_matrix, axis=0)
        top_indices = np.argsort(feature_variance)[-top_features:]
        df_subset = df.iloc[:, top_indices]
        print(f"\n📋 Top {top_features} features (por variância):")
        print(df_subset.round(3))
    else:
        print(f"\n📋 Todas as features:")
        print(df.round(3))
    
    return df

# Mostrar as três representações
df_binary = mostrar_matriz(bow_binary_matrix, bow_binary, "🎒 MATRIZ BINÁRIA", 8)
df_count = mostrar_matriz(bow_count_matrix, bow_count, "🎒 MATRIZ DE CONTAGEM", 8)  
df_tfidf = mostrar_matriz(tfidf_matrix, tfidf, "🎒 MATRIZ TF-IDF", 8)

# Comparar esparsidade
print(f"\n📊 COMPARAÇÃO DE ESPARSIDADE:")
print(f"Binary BoW: {(1.0 - bow_binary_matrix.nnz / (bow_binary_matrix.shape[0] * bow_binary_matrix.shape[1])):.3f}")
print(f"Count BoW:  {(1.0 - bow_count_matrix.nnz / (bow_count_matrix.shape[0] * bow_count_matrix.shape[1])):.3f}")
print(f"TF-IDF:     {(1.0 - tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1])):.3f}")

# Análise das palavras mais importantes no TF-IDF
print(f"\n🔝 PALAVRAS COM MAIOR TF-IDF MÉDIO:")
feature_names = tfidf.get_feature_names_out()
mean_tfidf = np.mean(tfidf_matrix.toarray(), axis=0)
top_indices = np.argsort(mean_tfidf)[-10:]

for idx in reversed(top_indices):
    print(f"{feature_names[idx]:12}: {mean_tfidf[idx]:.4f}")
